In [1]:
import json
import requests
from datetime import datetime, timedelta

In [2]:
config_file = "config.json"

In [3]:
# Funcion para leer configuracion desde JSON
def load_config(config_file):
    with open(config_file, "r") as file:
        return json.load(file)

In [4]:
# Función para calcular fechas de filtro
def calculate_date_filters():
    now = datetime.utcnow()
    twenty_four_hours_ago = now - timedelta(hours=24)
    return {
        "now": now.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "24_hours_ago": twenty_four_hours_ago.strftime("%Y-%m-%dT%H:%M:%SZ")
    }

In [5]:
# Función para obtener el token de autenticación
def get_token_PPDM (ppdm_instance):
    url = f'https://{ppdm_instance}:8443/api/v2/login'
    headers = {
        'Content-Type': 'application/json'
    }

    # Credenciales desde variables de entorno
    data = {
        username : 'DCOapi',
        password : 'Password123!'
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(data), verify=False)
        if response.status_code == 200:
            response_json = response.json()
            access_token = response_json.get('access_token')
            refresh_token = response_json.get('refresh_token')
            return access_token, refresh_token
        else:
            print(f"Error al obtener el token: {response.status_code}")
            return None, None
    except requests.RequestException as e:
        print(f"Error en la solicitud de autenticación: {e}")
        return None, None

In [6]:
# Construcción dinámica del endpoint con los filtros
def prepare_endpoint(instance, base_url, filters, fields):
    date_filters = calculate_date_filters()
    filter_expression = filters["filter_expression"].replace("24_hours_ago", date_filters["24_hours_ago"]).replace("now", date_filters["now"])
    endpoint = f"https://{instance}:8443{base_url}?filter_expression={filter_expression}&fields={','.join(fields)}"
    print(endpoint)
    return endpoint

In [7]:
# Función para hacer una llamada a la API
def fetch_data(endpoint, headers):
    response = requests.get(endpoint, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error al obtener datos de la API: {response.status_code}")
        return None

In [8]:
# Función para guardar datos en JSON
def save_json(data, system, instance, query_name):
    output_file = f"{system}-{instance}-{query_name}.json"
    with open(output_file, "w") as file:
        json.dump(data, file, indent=4)
    print(f"Datos guardados en: {output_file}")

In [9]:
# Función auxiliar para procesar cada sistema y sus queries
def process_endpoint(system, instance, endpoint_name, api_config):
    base_url = api_config["endpoint"]
    headers = api_config.get("headers", {})  # Headers como el "Authorization"
    endpoint = prepare_endpoint(instance, base_url, api_config["filters"], api_config["fields"])
    data = fetch_data(endpoint, headers)
    
    if data:
        save_json(data, system, instance, endpoint_name)

In [15]:
# Función principal
def main():
    print(config_file)
    config = load_config()
    print(config)

    for system, instances in config["systems"].items():
        for instance in instances:
            # Obtener token de autenticación
            access_token, _ = get_token_PPDM(instance)
            print(access_token)
            if not access_token:
                print(f"Error: no se pudo obtener el token para {instance}.")
                continue

            # Realizar llamadas según el sistema y guardar los resultados
            if system == "PPDM":
                data = fetch_activities_not_ok(instance)
                if data:
                    save_json(data, system, instance, "activities_not_ok")
                
                data = fetch_job_group_activities(instance)
                if data:
                    save_json(data, system, instance, "job_group_activities")
                
                # Obtener un token de autenticación para las llamadas que lo requieran
                access_token = "some_token_obtained_from_login"  # Aquí debería ir la lógica para obtener el token
                data = fetch_system_health_issues(instance, access_token)
                if data:
                    save_json(data, system, instance, "system_health_issues")
            # Aquí se podrían añadir más sistemas según se necesiten

In [16]:
if __name__ == "__main__":
    main()

config.json


TypeError: load_config() missing 1 required positional argument: 'config_file'